## LGE DS Mini Project 4team spaceship titanic

- ML project를 위해서 사용하는 템플릿 코드를 만듭니다.

1. **필요한 라이브러리와 데이터를 불러옵니다.**


2. **EDA를 수행합니다.** 이 때 EDA의 목적은 풀어야하는 문제를 위해서 수행됩니다.


3. **전처리를 수행합니다.** 이 때 중요한건 **feature engineering**을 어떻게 하느냐 입니다.


4. **데이터 분할을 합니다.** 이 때 train data와 test data 간의 분포 차이가 없는지 확인합니다.


5. **학습을 진행합니다.** 어떤 모델을 사용하여 학습할지 정합니다. 성능이 잘 나오는 GBM을 추천합니다.


6. **hyper-parameter tuning을 수행합니다.** 원하는 목표 성능이 나올 때 까지 진행합니다. 검증 단계를 통해 지속적으로 **overfitting이 되지 않게 주의**하세요.


7. **최종 테스트를 진행합니다.** 데이터 분석 대회 포맷에 맞는 submission 파일을 만들어서 성능을 확인해보세요.

## 1. 라이브러리, 데이터 불러오기

In [ ]:
# 설치에 필요한 라이브러리들이 있다면 모두 적어둡니다. anaconda에 기본적으로 설치되지 않은 라이브러리들을 적어두세요.
#!pip install lightgbm optuna

In [ ]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 모델들, 성능 평가
# (저는 일반적으로 정형데이터로 머신러닝 분석할 때는 이 2개 모델은 그냥 돌려봅니다. 특히 RF가 테스트하기 좋습니다.)
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from lightgbm.sklearn import LGBMRegressor

# 상관관계 분석, VIF : 다중공선성 제거
from statsmodels.stats.outliers_influence import variance_inflation_factor

# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import KFold
from functools import partial

# hyper-parameter tuning을 위한 라이브러리, optuna
import optuna

sns.set(style='darkgrid', font_scale=1.4)

In [ ]:
# flag setting
feature_reducing = True

feature_new_cabin = True
feature_new_expense_total = True
feature_new_expense_bool = False


In [ ]:
# 데이터를 불러옵니다.
train = pd.read_csv("../input/spaceship-titanic/train.csv")
test = pd.read_csv('../input/spaceship-titanic/test.csv')

In [ ]:
train.info()

## 2. EDA 수행


## 데이터의 형태를 파악해 보자 
- PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.

- HomePlanet - The planet the passenger departed from, typically their planet of permanent residence.

- CryoSleep - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.

- Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.
 
 *** deck num side로 나눠야 겠다. ***

- Destination - The planet the passenger will be debarking to.

- Age - The age of the passenger.

- VIP - Whether the passenger has paid for special VIP service during the voyage.

- RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.

  *** 사용한 금액이니까 데이터 분포(각 서비스의 가격을 보고) 합치면 어떨지? ***
  

- Name - The first and last names of the passenger.

- Transported - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.

In [ ]:
## On your Own
# 1. 데이터 크기 확인
train.info() # 8693 x 14 , 892KB
print("\n"+"-----"*20+"\n")

print(train.head())

print("\n"+"-----"*20+"\n")
train.describe()


#### 인사이트 (Train Data)

1. 전체 8693개 Row중 중 PassengerId, Transported 빼고 모든게 결측치가 있다. (v)

2. categorical feature: "HomePlanet", " CryoSleep", "Cabin" , "Destination", "VIP"

3. luxury amenities (RoomService, FoodCourt, ShoppingMall, Sap, VRDeck)의 중앙값이 0이다.

In [ ]:
# 1. target distribution
# Figure size
plt.figure(figsize=(6,6))

# Pie plot
train['Transported'].value_counts().plot.pie(explode=[0.1,0.1], autopct='%1.1f%%', shadow=True, textprops={'fontsize':16}).set_title("Target distribution")

In [ ]:
# 2. 상관관계 분석
corr = train.drop(columns=["PassengerId"]).corr()
plt.figure(figsize=(8, 8))
sns.heatmap(corr, annot=True)

#### 인사이트 (상관관계 분석) 

1. 잘 모르겠다. (Numerical feature에 대한 상관관계가 있을까??) -> 제일 큰 값이 0.2??

In [ ]:
# 전체 output 분포 

sns.histplot(data=train, x="Age", bins=10, hue="Transported", multiple="stack")
sns.displot(data=train,kind="kde", x="Age", hue="Transported")

######## 다른 곳의 Data 참고 
# Figure size
plt.figure(figsize=(10,4))

# Histogram
sns.histplot(data=train, x='Age', hue='Transported', binwidth=1, kde=True)

# Aesthetics
plt.title('Age distribution')
plt.xlabel('Age (years)')

#### 인사이트 - 데이타 분석(Age)

1. 나이가 어리면 Transported 될 확률이 높다. 나이대를 구분해서 넣으면 어떨까??? 

In [ ]:
# Categorical features
cat_feats=['HomePlanet', 'CryoSleep', 'Destination', 'VIP']

# Plot categorical features
fig=plt.figure(figsize=(10,16))
for i, var_name in enumerate(cat_feats):
    ax=fig.add_subplot(4,1,i+1)
    sns.countplot(data=train, x=var_name, axes=ax, hue='Transported')
    ax.set_title(var_name)
fig.tight_layout()  # Improves appearance a bit
plt.show()

#### 인사이트 (Categorical Features)

1. 모두 어느정도(?) 의미는 있어 보임 encoding을 하여 사용 할것 

2. (주의)Test에서 나오는 Categorical Feature 의 값의 범위가 같은지 Check  (v) 

In [ ]:
##질문! 위와 같이 하면 train2를 바꿔도 train 이 바뀐다. 
#train2=train.copy

# plt.figure(figsize=(10,8))
# plt.ylim(0,50)
# sns.histplot(data=train2, x='TotalSpend',bins=10, hue='Transported', binwidth=1)
# plt.show()

pd.pivot_table(data=train, index=["TotalSpend"], values=["Transported"], aggfunc=["mean","count"])
#craryosleep 사람은 뺀다??

#### 인사이트 (luxury amenities)

1. '0'을 쓴 사람들이 많다. 

2. (주의)Test에서 나오는 Categorical Feature 의 값의 범위가 같은지 Check  (v) 

In [ ]:
# 아래와 같이 분석했었는데, 숫자로 나와서 알아보기 힘들었다. 

#pd.pivot_table(data=train, index=["Age"], values=["Transported"], aggfunc=["mean", "count","sum"])
#pd.pivot_table(data=train, index=["CryoSleep"], values=["Transported"], aggfunc=["mean","count"])
#pd.pivot_table(data=train, index=["Destination"], values=["Transported"], aggfunc=["mean","count"])
#pd.pivot_table(data=train, index=["VIP"], values=["Transported"], aggfunc=["mean","count"])

In [ ]:
# 큰 의미를 찾을 수 없었음.. 해석하기도 힘들었음 
# for x in train.drop(columns=["PassengerId", "Transported"]).columns:
#     plt.figure(figsize=(12))
#     temp1 = pd.pivot_table(data=train, index=[x], values=["Transported"], aggfunc=["mean"])
#     plt.xlabel(x, fontsize=15)
#     plt.ylabel("Trasnsported")
#     plt.plot(temp1)
#     plt.show()
# #temp1

## 3. 전처리


In [ ]:
print(train.columns)
print(test.columns)

In [ ]:
train

In [ ]:
test

In [ ]:
# 1. drop cols 상관 없어 보이는 Col을 지운다. 
#train = train.drop(columns='Name')

train = train.drop(columns=["Name", "PassengerId"])
test = test.drop(columns=["Name","PassengerId"])


# 2. 결측치 체크 : column 방향으로 먼저 체크.
print(train.columns[train.isnull().any()])  

for col in train.columns:
    if train[col].dtype == 'object':
        train[col] = train[col].fillna(train[col].mode()[0])  # 최빈값으로 처리 
        test[col] = test[col].fillna(train[col].mode()[0])    # 최빈값으로 처리 
    elif train[col].dtype == 'float64':
        train[col] = train[col].fillna(train[col].median())  #중위값으로 처리 
        test[col] = test[col].fillna(train[col].median())    #중위값으로 처리 
        
print(train.columns[train.isnull().any()])  # 결측치가 있는지 없는지 체크 


In [ ]:
# 3. New Feature 여부 
# 3-1 Cabin 
# Cabin 구분 - train 

if feature_new_cabin:
    train['Cabin_deck'] = train['Cabin'].apply(lambda x: x.split('/')[0])
    train['Cabin_number'] = train['Cabin'].apply(lambda x: x.split('/')[1]).astype(int)
    train['Cabin_side'] = train['Cabin'].apply(lambda x: x.split('/')[2])

    # Cabin 구분 - Test 
    test['Cabin_deck'] = test['Cabin'].apply(lambda x: x.split('/')[0])
    test['Cabin_number'] = test['Cabin'].apply(lambda x: x.split('/')[1]).astype(int)
    test['Cabin_side'] = test['Cabin'].apply(lambda x: x.split('/')[2])

    # Drop Cabin (we don't need it anymore)
    train.drop('Cabin', axis=1, inplace=True)
    test.drop('Cabin', axis=1, inplace=True)

## 3-2 Total Expense  
if feature_new_expense_total:
    train["TotalSpend"] = train["RoomService"] + train["FoodCourt"] + train["ShoppingMall"] + train["Spa"] + train["VRDeck"]
    test["TotalSpend"] = test["RoomService"] + test["FoodCourt"] + test["ShoppingMall"] + test["Spa"] + test["VRDeck"]
elif feature_new_expense_bool:
    train["TotalSpend"] = (train["RoomService"] + train["FoodCourt"] + train["ShoppingMall"] + train["Spa"] + train["VRDeck"]) > 0
    test["TotalSpend"] = (test["RoomService"] + test["FoodCourt"] + test["ShoppingMall"] + test["Spa"] + test["VRDeck"]) > 0

In [ ]:

# 4. Dtype이 Object인 Column처리 

for col in train.columns:
    if train[col].dtype == 'object':
        print('categorical encoding for ['+col+']\n')
        train[col] = pd.factorize(train[col])[0]
        test[col] = pd.factorize(test[col])[0]
        

In [ ]:
train

## 4. 데이터 분할

In [ ]:
train

In [ ]:
#2. Data Split 
from sklearn.model_selection import train_test_split

X = train.drop(columns="Transported")
y = train.Transported


# 트테트테
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)


## 5. 학습

In [ ]:
# 5-1 Training
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=10, random_state=42)
clf.fit(X_train, y_train) # training

In [ ]:
# 5-2 Evaluation 
from sklearn.metrics import accuracy_score

pred = clf.predict(X_train)
pred2 = clf.predict(X_val)

print("Training Acc : %.4f" % accuracy_score(y_train, pred))
print("Validation Acc : %.4f" % accuracy_score(y_val, pred2))

## 6. 하이퍼파라미터 튜닝


## 7. 최종 테스트 및 제출

In [ ]:
test

In [ ]:
#pediction 
X_test = test
result = clf.predict(X_test)
result

In [ ]:
submission = pd.read_csv('../input/spaceship-titanic/sample_submission.csv')
submission["Transported"] = result  # 정답을 채워줍니다.
submission

In [ ]:
#submission.reset_index(drop=True).to_csv("submission.csv", index=False)

submission.to_csv('submission.csv', index=False)
#submission

In [ ]:
# import os

# file_path = './submission1.csv'
# os.remove(file_path)